<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- replace_abbreviations
- remove url 
- get the emoji and replace them as words
- remove stop words
- keep only english letters
- stemming

In [3]:
# !pip install emoji
# !pip install nltk

  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5a92f91dcf134fc24011694b0e97f509edb85899e2f6e4aeb666514621a3bb94
  Stored in directory: c:\users\trist\appdata\local\pip\cache\wheels\5e\8c\80\c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


In [1]:
import emoji
from nltk.corpus import stopwords
import re
import nltk
import json
import pandas as pd
from utils import timer

In [ ]:
nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopword = stopwords.words('english')


@timer('ms')
def clean_tweet(content):
    # replace_abbreviations
    content = content.lower()
    content = re.sub(r"won't", "will not", content)
    content = re.sub(r"can't", "can not", content)
    content = re.sub(r"cannot", "can not", content)
    content = re.sub(r"n't", " not", content)
    content = re.sub(r"'re", " are", content)
    content = re.sub(r"'s", " is", content)
    content = re.sub(r"'d", " would", content)
    content = re.sub(r"'ll", " will", content)
    content = re.sub(r"'t", " not", content)
    content = re.sub(r"'ve", " have", content)
    content = re.sub(r"'m", " am", content)

    content = re.sub(r'@[A-Za-z0-9_]+', '', content) # remove twitter ID
    # remove url 
    ## http, https
    content = re.sub(r'https?://[^ ]+', '', content) 
    ## www.
    content = re.sub(r'www.[^ ]+', '', content)
    # get the emoji and replace them as words
    emojis = emoji.distinct_emoji_list(content)
    for e in emojis:
        content = re.sub(e, emoji.demojize(e), content)
    content = re.sub('\w+\d+\w+', '', content) # remove the word contains numbers
    content = re.sub(r'[:_.!+-=——,$%^\.\?\\~\"\'@#$%&*<>{}\[\]()/]', ' ', content) # remove punctuation
    content = re.sub(r"\s+", " ", content) # conver multiple spaces as a single space
    content = content.strip()
    # remove stop words and keep only english letters
    content = [c for c in content.split(' ') if c not in stopword and c.isalpha()]
    # do stemming
    content = [stemmer.stem(token) for token in content]
    
    return content

In [ ]:
@timer('ms')
def sort_by_time(df):
    

In [ ]:
@timer('ms')
def clean_data(json_file):
  """
  json_file: 'train_reply.json'
  """
    with open(json_file, 'r+') as file:
        content=file.read()
    content=json.loads(content)
    df = pd.DataFrame(content)
    df = df.T
    df['text'] = df['text'].apply(lambda x: clean_tweet(x))
    return df


### Sort raw data by time

In [4]:
@timer('ms')
def sort_by_time(raw_file, json_file):
    with open(raw_file) as file:
        ids = file.readlines()

    with open(json_file, 'r+') as file:
        content = file.read()
        content=json.loads(content)
        df = pd.DataFrame(content)
        df = df.T

    save_name = raw_file[:-4] + '_sorted.txt'
    with open(save_name, 'w') as file:
        date = pd.Series(pd.DatetimeIndex(df.iloc[:, 6]), index=df.index)
        df.drop(['created_at'], axis=1, inplace=True)
        df['time'] = date

        for id_ in ids:
            ids_ = id_.strip().split(',')
            source_id = ids_[0]
            file.write(source_id)
            if len(ids_) > 1:
                reply_ids = ids_[1:]
                reply_ids[-1] = reply_ids[-1].replace('\n', '')
                valid_ids = [index for index in reply_ids if index in df.index]
                sorted_replies = df.loc[valid_ids].sort_values(by='time')
                if len(valid_ids) > 0:
                    file.write(',')

                for i, index in enumerate(sorted_replies.index):
                    file.write(index)
                    if i != len(sorted_replies.index) - 1:
                        file.write(',')

            file.write('\n')

In [5]:
raw_files = ['train.data.txt', 'dev.data.txt']
json_files = ['./filtered data/train_reply.json', './filtered data/dev_reply.json']
for raw_file, json_file in zip(raw_files, json_files):
    sort_by_time(raw_file, json_file)

Call sort_by_time in 2248.7099170684814 ms
Call sort_by_time in 746.5729713439941 ms


In [6]:
with open(raw_files[0]) as file:
    train = file.readlines()
with open('train.data_sorted.txt') as file:
    s = file.readlines()
assert len(train) == len(s)